In [17]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the Excel file
file_path =r"C:\Users\ScarboroC\Downloads\fb_combined_view (1).xlsx"
xls = pd.ExcelFile(file_path, engine='openpyxl')

# Load full dataset with necessary columns
columns = [
    'order_id', 'name', 'price', 'type', 'id', 'quantity', 'tax', 'total', 'original_subtotal',
    'status', 'vendor_id', 'vendor_name', 'venue_name', 'payment_type',
    'order_date', 'processed_date', 'event_date', 'event_name', 'event_time', 'season_name'
]
df = pd.read_excel(xls, sheet_name=0, usecols=columns)

# Standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_")

# Clean and parse order_date and event_date
# Ensure order_date is a valid time string

df["order_date"] = df["order_date"].astype(str).str.strip()
df["order_date"] = df["order_date"].apply(lambda x: x if ":" in x else "00:00:00")

# Parse event_date to datetime

df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce")
df["event_date"] = df["event_date"].fillna(pd.to_datetime("2024-01-01"))


#Adjust event_date for night 1 and 2 occurences 
df.loc[df["event_name"].str.contains("Night 1", case=False, na=False), "event_date"] = df.loc[df["event_name"].str.contains("Night 1", case=False, na=False), "event_date"] - pd.Timedelta(days=1)


# Combine event_date and order_time into order_datetime

df["order_datetime"] = df["event_date"].astype(str) + " " + df["order_date"]
df["order_datetime"] = pd.to_datetime(df["order_datetime"], errors="coerce", format="%Y-%m-%d %H:%M:%S")

# Drop invalid rows and duplicates
df = df.dropna(subset=["order_datetime", "total", "price", "original_subtotal"])
df = df.drop_duplicates()

# Recalculate quantity

df["quantity"] = df.apply(
    lambda row: round(row["original_subtotal"] / row["price"]) if row["price"] not in [0, None] else 0,
    axis=1
)

# Create time-based features for analysis
df["minute"] = df["order_datetime"].dt.minute

# --- Insights --- #

# 1. Top-Selling Items by Revenue
top_items = df.groupby("name")["total"].sum().sort_values(ascending=False).head(10)
print("Top Selling Items:\n", top_items)

# 2. Revenue by Event
revenue_by_event = df.groupby("event_name")["total"].sum().sort_values(ascending=False).head(10)
print("\nRevenue by Event:\n", revenue_by_event)

# 3. Peak Ordering Times by Minute (using order_id count)
peak_minutes = df.groupby("minute")["order_id"].count().sort_values(ascending=False)
print("\nPeak Ordering Times by Minute:\n", peak_minutes.head(10))

# 4. Vendor Performance
top_vendors = df.groupby("vendor_name")["total"].sum().sort_values(ascending=False).head(10)
print("\nTop Vendors:\n", top_vendors)

df.to_csv("lafc_cleaned_data.csv", index=False)

Top Selling Items:
 name
Aquafina 20oz                 1.139073e+06
Draft, Modelo 20oz            7.990107e+05
Can, Modelo 24oz              7.193545e+05
Draft, Pacifico 20oz          6.202103e+05
Can, Pacifico 24oz            5.943606e+05
Chicken Tenders & Fries       4.489705e+05
MarioChelada Upcharge         4.354217e+05
Pepsi 20oz                    3.706303e+05
Can, Truly Wild Berry 24oz    3.567193e+05
1800 Las Margaritas           3.559777e+05
Name: total, dtype: float64

Revenue by Event:
 event_name
Crawford vs. Madrimov       641141.4365
KoRn                        463171.9585
Grupo Firme                 416216.8710
Carin Leon                  395291.2378
Foo Fighters - Night 2      390726.1625
Kane Brown                  390400.7335
Foo Fighters - Night 1      385670.5688
ODESZA - Night 2            339050.0576
Los Temerarios - Night 1    334391.8615
Los Temerarios - Night 2    301165.7191
Name: total, dtype: float64

Peak Ordering Times by Minute:
 minute
38    5864
41    5